In [161]:
import tensorflow as tf
import numpy as np
import pandas as pd
from matplotlib import plt
from tensorflow.python.framework import ops
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data",one_hot=True)
def load_data(mnist):
    x_train=mnist.train.images
    y_train=mnist.train.labels
    x_test=mnist.test.images
    y_test=mnist.test.labels
    return x_train,y_train,x_test,y_test

ImportError: cannot import name 'plt'

In [89]:
x_train,y_train,x_test,y_test = load_data(mnist)
print(x_test.shape,y_test.shape,x_train.shape,y_train.shape)

(10000, 784) (10000, 10) (55000, 784) (55000, 10)


In [90]:
def create_placeholder(nx,ny):
    X=tf.placeholder(dtype='float',shape=[nx,None])
    Y=tf.placeholder(dtype='float',shape=[ny,None])
    return X,Y

In [91]:
def initialize_parameter(L,nL):  #L no of hidden layers 1 extra softmax layer
    parameters = {}
    seed=1
    for i in range(1,L+1):
        w=tf.get_variable("W"+str(i),[nL[i],nL[i-1]],initializer=tf.contrib.layers.xavier_initializer(seed=seed))
        b=tf.get_variable("b"+str(i),[nL[i],1],initializer=tf.zeros_initializer())
        parameters.update({'W'+str(i) : w})
        parameters.update({'b'+str(i) : b})
        seed=seed+1
        
    return parameters 
        
    
    

In [92]:
tf.reset_default_graph()
with tf.Session() as sess:
    parameters = initialize_parameter(3,[4,8,9,7])
    print(str(parameters))
    

{'W1': <tf.Variable 'W1:0' shape=(8, 4) dtype=float32_ref>, 'b1': <tf.Variable 'b1:0' shape=(8, 1) dtype=float32_ref>, 'W2': <tf.Variable 'W2:0' shape=(9, 8) dtype=float32_ref>, 'b2': <tf.Variable 'b2:0' shape=(9, 1) dtype=float32_ref>, 'W3': <tf.Variable 'W3:0' shape=(7, 9) dtype=float32_ref>, 'b3': <tf.Variable 'b3:0' shape=(7, 1) dtype=float32_ref>}


In [160]:
def forward_prop(L,X,parameters):
    prev_act=X
    for i in range(1,L+1):
        W=parameters['W'+str(i)]
        b=parameters['b'+str(i)]
        curr_z=tf.matmul(W,prev_act)+b 
        curr_act=tf.nn.tanh(curr_z)
        prev_act=curr_act
        
    ZL=tf.nn.softmax(prev_act)
    return ZL
      
    

In [94]:
tf.reset_default_graph()

with tf.Session() as sess:
    X, Y = create_placeholder(784,10)
    parameters = initialize_parameter(3,[784,25,17,10])
    ZL = forward_prop(3,X, parameters)
    print("ZL = " + str(ZL))

ZL = Tensor("Softmax:0", shape=(10, ?), dtype=float32)


In [142]:
def compute_cost(ZL,Y):
    logits=tf.transpose(ZL)
    labels=tf.transpose(Y)
    cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels,logits=logits))
    return cost

In [136]:





tf.reset_default_graph()

with tf.Session() as sess:
    X, Y = create_placeholder(784,10)
    parameters = initialize_parameter(3,[784,25,17,10])
    ZL = forward_prop(3,X, parameters)
    print(compute_cost(ZL,Y))

Tensor("Mean:0", shape=(), dtype=float32)


SyntaxError: non-default argument follows default argument (<ipython-input-100-d32201f85246>, line 1)

In [162]:
def model(x_train, y_train, x_test, y_test, learning_rate = 0.008,
          num_epochs = 120, minibatch_size =32, print_cost = True):
    
    ops.reset_default_graph()                         # to be able to rerun the model without overwriting tf variables                                 # to keep consistent results
    ( m,n_x) = x_train.shape                          # (n_x: input size, m : number of examples in the train set)
    n_y = y_train.shape[1]                            # n_y : output size
    costs = []                                        # To keep track of the cost
    
    # Create Placeholders of shape (n_x, n_y)
    ### START CODE HERE ### (1 line)
    X, Y = create_placeholder(n_x, n_y)
    ### END CODE HERE ###

    # Initialize parameters
    ### START CODE HERE ### (1 line)
    parameters = initialize_parameter(5,[784,50,40,30,20,10])
    ### END CODE HERE ###
    
    # Forward propagation: Build the forward propagation in the tensorflow graph
    ### START CODE HERE ### (1 line)
    ZL= forward_prop(5,X, parameters)
    ### END CODE HERE ###
    
    # Cost function: Add cost function to tensorflow graph
    ### START CODE HERE ### (1 line)
    cost = compute_cost(ZL, Y)
    ### END CODE HERE ###
    
    # Backpropagation: Define the tensorflow optimizer. Use an AdamOptimizer.
    ### START CODE HERE ### (1 line)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    ### END CODE HERE ###
    
    # Initialize all the variables
    init = tf.global_variables_initializer()

    # Start the session to compute the tensorflow graph
    with tf.Session() as sess:
        
        # Run the initialization
        sess.run(init)
        
        # Do the training loop
        for epoch in range(num_epochs):

            epoch_cost = 0.                       # Defines a cost related to an epoch
            num_minibatches = int(m / minibatch_size) # number of minibatches of size minibatch_size in the train set

            for minibatch in range(num_minibatches):

                # Select a minibatch
                minibatch_X, minibatch_Y = mnist.train.next_batch(minibatch_size) 
                # IMPORTANT: The line that runs the graph on a minibatch.
                # Run the session to execute the "optimizer" and the "cost", the feedict should contain a minibatch for (X,Y).
                ### START CODE HERE ### (1 line)
                _ , minibatch_cost = sess.run([optimizer, cost], feed_dict={X: minibatch_X.T, Y: minibatch_Y.T})
                ### END CODE HERE ###
                
                epoch_cost += minibatch_cost / num_minibatches

            # Print the cost every epoch
            if print_cost == True and epoch % 10 == 0:
                print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
            if print_cost == True and epoch % 5 == 0:
                costs.append(epoch_cost)
                
        # plot the cost
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per tens)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()

        # lets save the parameters in a variable
        parameters = sess.run(parameters)
        print("Parameters have been trained!")

        # Calculate the correct predictions
        correct_prediction = tf.equal(tf.argmax(ZL), tf.argmax(Y))

        # Calculate accuracy on the test set
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

        print("Train Accuracy:", accuracy.eval({X: X_train, Y: Y_train}))
        print("Test Accuracy:", accuracy.eval({X: X_test, Y: Y_test}))
        
        return parameters

In [ ]:
parameters = model(x_train, y_train, x_test, y_test)

Cost after epoch 0: 2.221756
Cost after epoch 10: 2.216487
Cost after epoch 20: 2.216563
Cost after epoch 30: 2.218062
Cost after epoch 40: 2.216905
